# Tiny GPT
Build a small GPT model using PyTorch (in mac)

## Check PyTorch instance

_Notes:_

**MPS (Metal Performance Shaders)**
* Metal Performance Shaders is an Apple framework of highly optimized GPU shaders for image processing, linear algebra, and neural networks on top of Metal.
* PyTorch, diffusers, and other ML libraries expose an mps device to offload tensor operations to the GPU via Metal/MPS on Apple silicon.


In [4]:
import torch

print(f"Torch version: {torch.__version__} and MPS availability is: {torch.backends.mps.is_available()}")

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)

print(f"Default device set to: {torch.get_default_device()}")
print(f"Is MPS built: {torch.backends.mps.is_built()}")

Torch version: 2.9.1 and MPS availability is: True
Default device set to: mps:0
Is MPS built: True


## Mathematical Explanation

A LLM model only understands numbers - based on which it calculates the probability. The model selects the next one word that has got the highest probability.
$$
\begin{aligned}
P(w_1,w_2,w_3,......,w_n) = \prod_{t=1}^{n}P(w_t|w_1,w_2,......,w_{t-1}) \\
\text{where, } w_i \text{ represents a token}
\end{aligned}
$$
E.g., suppose,
* we have the vocabulary (training set): [`very`, `tea`, `hot`, `is`, `the`]
* the first 2 words fed to the model is: [`the`, `tea`]
* the model will use the above formula of _Chain Probability_ to predict the next likely word.
    * So, it will execute `P('is'|'the','tea')`. This reads as probability of `is` given the previous words are `the` and `tea`. Similarly it will execute `P('hot'|'the','tea')` and so on. The one with the highest probability wins.
    * For a set of 4 tokens, the overall probability is calculated as
$$
\begin{aligned}
P(w_1,w_2,w_3,w_4) = P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times P(w_4|w_1, w_2, w_3)
\end{aligned}
$$

## Training Data

In [5]:
corpus = [
    "hello friends how are you",
    "the tea is very hot",
    "my name is Sushovan",
    "the roads of Kolkata are busy",
    "it is raining in Mumbai",
    "the train is late again",
    "i love eating samosas and drinking tea",
    "holi is my favorite festival",
    "diwali brings lights and sweets",
    "india won the cricket match"
]

## Tokenize Data for the model
_Note_: Instead of using a tokenizer library (e.g. `BPE` - `Byte Pair Encoding` used in GPT or `SentencePiece` used in LLAMA), we shall be building a custom tokenizer to understand the concepts.

### Put end marker and concatenate the corpus

In [6]:
text = " ".join([s + " <END>" for s in corpus])
print(text)

hello friends how are you <END> the tea is very hot <END> my name is Sushovan <END> the roads of Kolkata are busy <END> it is raining in Mumbai <END> the train is late again <END> i love eating samosas and drinking tea <END> holi is my favorite festival <END> diwali brings lights and sweets <END> india won the cricket match <END>


### Extract the words to build the vocabulary

In [7]:
words = list(set(text.split()))
vocab_size = len(words)
print(f"Vocabulary: {words}")
print(f"Vocabulary Size: {vocab_size}")

Vocabulary: ['of', 'lights', 'it', 'love', 'name', 'you', 'are', 'in', 'i', 'Kolkata', 'and', 'diwali', 'match', 'how', 'train', 'tea', 'sweets', '<END>', 'samosas', 'hello', 'very', 'raining', 'the', 'is', 'festival', 'Mumbai', 'drinking', 'holi', 'favorite', 'roads', 'busy', 'hot', 'eating', 'won', 'india', 'late', 'my', 'again', 'friends', 'Sushovan', 'cricket', 'brings']
Vocabulary Size: 42


### Build the word index

In [8]:
word2idx = {w: idx for idx, w in enumerate(words)}
print(f"Words to Index:  {word2idx}")

idx2word = {idx: w for w, idx in word2idx.items()}
print(f"Index to Words:  {idx2word}")

Words to Index:  {'of': 0, 'lights': 1, 'it': 2, 'love': 3, 'name': 4, 'you': 5, 'are': 6, 'in': 7, 'i': 8, 'Kolkata': 9, 'and': 10, 'diwali': 11, 'match': 12, 'how': 13, 'train': 14, 'tea': 15, 'sweets': 16, '<END>': 17, 'samosas': 18, 'hello': 19, 'very': 20, 'raining': 21, 'the': 22, 'is': 23, 'festival': 24, 'Mumbai': 25, 'drinking': 26, 'holi': 27, 'favorite': 28, 'roads': 29, 'busy': 30, 'hot': 31, 'eating': 32, 'won': 33, 'india': 34, 'late': 35, 'my': 36, 'again': 37, 'friends': 38, 'Sushovan': 39, 'cricket': 40, 'brings': 41}
Index to Words:  {0: 'of', 1: 'lights', 2: 'it', 3: 'love', 4: 'name', 5: 'you', 6: 'are', 7: 'in', 8: 'i', 9: 'Kolkata', 10: 'and', 11: 'diwali', 12: 'match', 13: 'how', 14: 'train', 15: 'tea', 16: 'sweets', 17: '<END>', 18: 'samosas', 19: 'hello', 20: 'very', 21: 'raining', 22: 'the', 23: 'is', 24: 'festival', 25: 'Mumbai', 26: 'drinking', 27: 'holi', 28: 'favorite', 29: 'roads', 30: 'busy', 31: 'hot', 32: 'eating', 33: 'won', 34: 'india', 35: 'late', 3

### Convert to tensor
Replace each word in text with the corresponding index and fed that to a tensor

In [9]:
data = torch.tensor([word2idx[idx] for idx in text.split()], dtype=torch.long)
print(f"Tensor Data: {data}")
print(f"Data Shape: {data.shape}")

Tensor Data: tensor([19, 38, 13,  6,  5, 17, 22, 15, 23, 20, 31, 17, 36,  4, 23, 39, 17, 22,
        29,  0,  9,  6, 30, 17,  2, 23, 21,  7, 25, 17, 22, 14, 23, 35, 37, 17,
         8,  3, 32, 18, 10, 26, 15, 17, 27, 23, 36, 28, 24, 17, 11, 41,  1, 10,
        16, 17, 34, 33, 22, 40, 12, 17], device='mps:0')
Data Shape: torch.Size([62])


## Define Parameters

`block_size`: also known as context_length. It means that how many previous words the llm would refer, to predict the next word

In [10]:
block_size = 6

`embedding_dim`: Embedding Dimension

Each word in the tensor will be represented in the llm model as a dimensional vector of a defined size. Initially, random numbers are generate for each value in the dimensional vector. The llm uses this vector to predict the next word.

In [11]:
embedding_dim = 32

In [12]:
n_heads = 2  # number of Multi-head attention layer

In [13]:
n_layers = 2  # number of transformer blocks to use

In [14]:
lr = 1e-3  # learning rate

In [15]:
epochs = 1500  # number of training iterations

## Define batch function

`batch_size`: This indicates the number of sequences (or sentences) for the model to consider

In [16]:
def get_batch(batch_size=16):
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    # the above function gives 16 examples of random sentences
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return ix, x, y


rand_int, input_batch, output_bath = get_batch()
print(f"Data: {data}")
print(f"Batch Index: {rand_int}")
print(f"input_batch shape: {input_batch.data}")
print(f"output_bath shape: {output_bath.data}")

Data: tensor([19, 38, 13,  6,  5, 17, 22, 15, 23, 20, 31, 17, 36,  4, 23, 39, 17, 22,
        29,  0,  9,  6, 30, 17,  2, 23, 21,  7, 25, 17, 22, 14, 23, 35, 37, 17,
         8,  3, 32, 18, 10, 26, 15, 17, 27, 23, 36, 28, 24, 17, 11, 41,  1, 10,
        16, 17, 34, 33, 22, 40, 12, 17], device='mps:0')
Batch Index: tensor([52, 20, 22,  0, 53, 39, 41, 27, 50, 31, 34, 34, 25, 19, 29,  8],
       device='mps:0')
input_batch shape: tensor([[ 1, 10, 16, 17, 34, 33],
        [ 9,  6, 30, 17,  2, 23],
        [30, 17,  2, 23, 21,  7],
        [19, 38, 13,  6,  5, 17],
        [10, 16, 17, 34, 33, 22],
        [18, 10, 26, 15, 17, 27],
        [26, 15, 17, 27, 23, 36],
        [ 7, 25, 17, 22, 14, 23],
        [11, 41,  1, 10, 16, 17],
        [14, 23, 35, 37, 17,  8],
        [37, 17,  8,  3, 32, 18],
        [37, 17,  8,  3, 32, 18],
        [23, 21,  7, 25, 17, 22],
        [ 0,  9,  6, 30, 17,  2],
        [17, 22, 14, 23, 35, 37],
        [23, 20, 31, 17, 36,  4]], device='mps:0')
output_b

In [17]:
import numpy as np

print(f"Decoded data: \n{[idx2word[i.item()] for i in data]}")
print(f"input stack words: \n{np.array([[idx2word[i.item()] for i in b] for b in input_batch])}")
print(f"output stack words: \n{np.array([[idx2word[i.item()] for i in b] for b in output_bath])}")

Decoded data: 
['hello', 'friends', 'how', 'are', 'you', '<END>', 'the', 'tea', 'is', 'very', 'hot', '<END>', 'my', 'name', 'is', 'Sushovan', '<END>', 'the', 'roads', 'of', 'Kolkata', 'are', 'busy', '<END>', 'it', 'is', 'raining', 'in', 'Mumbai', '<END>', 'the', 'train', 'is', 'late', 'again', '<END>', 'i', 'love', 'eating', 'samosas', 'and', 'drinking', 'tea', '<END>', 'holi', 'is', 'my', 'favorite', 'festival', '<END>', 'diwali', 'brings', 'lights', 'and', 'sweets', '<END>', 'india', 'won', 'the', 'cricket', 'match', '<END>']
input stack words: 
[['lights' 'and' 'sweets' '<END>' 'india' 'won']
 ['Kolkata' 'are' 'busy' '<END>' 'it' 'is']
 ['busy' '<END>' 'it' 'is' 'raining' 'in']
 ['hello' 'friends' 'how' 'are' 'you' '<END>']
 ['and' 'sweets' '<END>' 'india' 'won' 'the']
 ['samosas' 'and' 'drinking' 'tea' '<END>' 'holi']
 ['drinking' 'tea' '<END>' 'holi' 'is' 'my']
 ['in' 'Mumbai' '<END>' 'the' 'train' 'is']
 ['diwali' 'brings' 'lights' 'and' 'sweets' '<END>']
 ['train' 'is' 'late' 'a

## Build the model

In [18]:
import torch.nn as nn
import torch.nn.functional as F
from transformer.transformer_utils import Block


class TinyGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.positional_embeddings = nn.Embedding(block_size, embedding_dim)
        self.blocks = nn.Sequential(*[Block(embedding_dim, block_size, n_heads) for _ in range(n_layers)])

        self.ln_f = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_embeddings = self.token_embeddings(idx)
        position_embeddings = self.positional_embeddings(torch.arange(T, device=idx.device))

        x = token_embeddings + position_embeddings
        x = self.blocks(x)
        x = self.ln_f(x)

        logits = self.head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B * T, C), targets.view(B * T))

        return logits, loss

    def generate(self, idx, max_tokens=10):
        for _ in range(max_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, 1)
            idx = torch.cat((idx, next_idx), dim=1)
        return idx


## Train the model

In [19]:
model = TinyGPT()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for step in range(epochs):
    _, xb, yb = get_batch()
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(f"Step: {step}, Loss: {loss.item():.4f}")
    elif step == epochs - 1:
        print(f"Step: {step}, Loss: {loss.item():.4f}")

Step: 0, Loss: 4.0531
Step: 100, Loss: 1.2058
Step: 200, Loss: 0.3526
Step: 300, Loss: 0.2506
Step: 400, Loss: 0.1945
Step: 500, Loss: 0.1063
Step: 600, Loss: 0.1407
Step: 700, Loss: 0.2601
Step: 800, Loss: 0.0954
Step: 900, Loss: 0.1371
Step: 1000, Loss: 0.0775
Step: 1100, Loss: 0.1871
Step: 1200, Loss: 0.0989
Step: 1300, Loss: 0.1579
Step: 1400, Loss: 0.0543
Step: 1499, Loss: 0.1471


## Execute the model

In [21]:
def execute_model(context):
    word_indexed = [word2idx[word] for word in context.split()]
    context = torch.tensor([word_indexed], dtype=torch.long)
    out = model.generate(context)

    print("\nGenerated Text:\n")
    print(" ".join(idx2word[int(i)] for i in out[0]))

In [25]:
execute_model("hello")


Generated Text:

hello friends how are you <END> the tea is very hot


In [26]:
execute_model("my name")


Generated Text:

my name is Sushovan <END> the roads of Kolkata are busy <END>
